### This Notebook is used to understand Aprioiri Algorithm

**Following Cell is used to load a test dataset containing 5 records. But You Should use your funcion to load the datset**

In [52]:
import pprint

def load_dataset():
    """Loads an example of market basket transactions for testing purposes.

    Returns
    -------
    A list (database) of lists (transactions). Each element of a transaction 
    is an item.
    """
    return [['Bread', 'Milk'], 
            ['Bread', 'Diapers', 'Beer', 'Eggs'], 
            ['Milk', 'Diapers', 'Beer', 'Coke'], 
            ['Bread', 'Milk', 'Diapers', 'Beer'], 
            ['Bread', 'Milk', 'Diapers', 'Coke']]

dataset = load_dataset() # list of transactions; each transaction is a list of items
D = map(set, dataset) # set of transactions; each transaction is a list of items
print(len(dataset))
pprint.pprint(dataset)
#print(dataset)

5
[['Bread', 'Milk'],
 ['Bread', 'Diapers', 'Beer', 'Eggs'],
 ['Milk', 'Diapers', 'Beer', 'Coke'],
 ['Bread', 'Milk', 'Diapers', 'Beer'],
 ['Bread', 'Milk', 'Diapers', 'Coke']]


In [53]:
def create_candidates(dataset, VERBOSE=False):
    """Creates a list of candidate 1-itemsets from a list of transactions.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    Returns
    -------
    The list of candidate itemsets (c1) passed as a frozenset (a set that is 
    immutable and hashable).
    """
    c1 = [] # list of all items in the database of transactions
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()

    if VERBOSE:
        # Print a list of all the candidate items.
        print ("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")

    # Map c1 to a frozenset because it will be the key of a dictionary.
    return map(set, c1)
    return(c1)
    #return C1

In [54]:
# Generate candidate itemsets.
C1 = create_candidates(dataset, VERBOSE=True) # candidate 1-itemsets

{Beer, Bread, Coke, Diapers, Eggs, Milk}


**Now a function is defined to prune the candidates from the database**

In [55]:
def support_prune(dataset, candidates, min_support, VERBOSE=True):
    """Returns all candidate itemsets that meet a minimum support threshold.

    By the apriori principle, if an itemset is frequent, then all of its 
    subsets must also be frequent. As a result, we can perform support-based 
    pruning to systematically control the exponential growth of candidate 
    itemsets. Thus, itemsets that do not meet the minimum support level are 
    pruned from the input list of itemsets (dataset).

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    candidates : frozenset
        The list of candidate itemsets.

    min_support : float
        The minimum support threshold.

    Returns
    -------
    retlist : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.
    """
    print("values inside function")
    pprint.pprint(dataset)
    print("candidates")
    print(candidates)
    sscnt = dict() # set for support counts
    num_items=0
    #for candidate in iter(candidates):
        #print(candidate)
    
    
    
    for tid in dataset:
        print("tid")
        print(tid)
        num_items+=1
        for can in iter(candidates):
            print("Here1")
            #print(next(can))
            if can.issubset(tid):
                print("Here 2")
                sscnt.setdefault(can,0)
                sscnt[can] += 1

    #num_items = 5 #len(dataset) # total number of transactions in the dataset
    retlist = [] # array for unpruned itemsets
    support_data = {} # set for support data for corresponding itemsets
    
    pprint.pprint(sscnt)
    for key in sscnt:
        # Calculate the support of itemset key.
        print(key)
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support

    # Print a list of the pruned itemsets.
    if VERBOSE:
        for kset in retlist:
            for item in kset:
                print ("{" + str(item) + "}")
        #print
        for key in sscnt:
            print ("" \
                + "{" \
                + "".join([str(i) + ", " for i in iter(key)]).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(support_data[key]))

    return retlist, support_data

**Now Test this Function Support_prune**

In [56]:
# Prune candidate 1-itemsets via support-based pruning to generate frequent 1-itemsets.
F1, support_data = support_prune(dataset, C1, 0.1, VERBOSE=True)
print('here 3')
print(F1)
print(support_data)

values inside function
[['Bread', 'Milk'],
 ['Bread', 'Diapers', 'Beer', 'Eggs'],
 ['Milk', 'Diapers', 'Beer', 'Coke'],
 ['Bread', 'Milk', 'Diapers', 'Beer'],
 ['Bread', 'Milk', 'Diapers', 'Coke']]
candidates
tid
['Bread', 'Milk']
Here1
Here1
Here 2


TypeError: unhashable type: 'set'

**Now We will Make two functions 'Apriori' and 'Apriori Gen'**

In [40]:
def apriori(dataset, min_support=0.1, VERBOSE=False):
    """Implements the Apriori algorithm.

    The Apriori algorithm will iteratively generate new candidate 
    k-itemsets using the frequent (k-1)-itemsets found in the previous 
    iteration.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    Returns
    -------
    F : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.


    """
    C1 = create_candidates(dataset)
    D = map(set, dataset)
    F1, support_data = support_prune(D, C1, min_support, VERBOSE=False) # prune candidate 1-itemsets
    F = [F1] # list of frequent itemsets; initialized to frequent 1-itemsets
    k = 2 # the itemset cardinality
    while (len(F[k - 2]) > 0):
        Ck = apriori_gen(F[k-2], k) # generate candidate itemsets
        Fk, supK = support_prune(D, Ck, min_support) # prune candidate itemsets
        support_data.update(supK) # update the support counts to reflect pruning
        F.append(Fk) # add the pruned candidate itemsets to the list of frequent itemsets
        k += 1

    if VERBOSE:
        # Print a list of all the frequent itemsets.
        for kset in F:
            for item in kset:
                print( "" \
                    + "{" \
                    + "".join(str(i) + ", " for i in iter(item)).rstrip(', ') \
                    + "}" \
                    + ":  sup = " + str(round(support_data[item], 3)))

    return F, support_data

In [41]:
def apriori_gen(freq_sets, k):
    """Generates candidate itemsets (via the F_k-1 x F_k-1 method).

    This operation generates new candidate k-itemsets based on the frequent 
    (k-1)-itemsets found in the previous iteration. The candidate generation 
    procedure merges a pair of frequent (k-1)-itemsets only if their first k-2 
    items are identical.

    Parameters
    ----------
    freq_sets : list
        The list of frequent (k-1)-itemsets.

    k : integer
        The cardinality of the current itemsets begin evaluated.

    Returns
    -------
    retlist : list
        The list of merged frequent itemsets.
    """
    retList = [] # list of merged frequent itemsets
    lenLk = len(freq_sets) # number of frequent itemsets
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(freq_sets[i])
            b=list(freq_sets[j])
            a.sort()
            b.sort()
            F1 = a[:k-2] # first k-2 items of freq_sets[i]
            F2 = b[:k-2] # first k-2 items of freq_sets[j]

            if F1 == F2: # if the first k-2 items are identical
                # Merge the frequent itemsets.
                retList.append(freq_sets[i] | freq_sets[j])

    return retList


#### Now Test Run the Apriori

In [42]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.02, VERBOSE=True)
print("this is F")
print(F)

frozenset({'Beer'})
frozenset({'Bread'})
frozenset({'Coke'})
frozenset({'Diapers'})
frozenset({'Eggs'})
frozenset({'Milk'})
{'Milk', 'Bread'}
{'Eggs', 'Diapers', 'Beer', 'Bread'}
{'Coke', 'Milk', 'Diapers', 'Beer'}
{'Milk', 'Diapers', 'Beer', 'Bread'}
{'Coke', 'Milk', 'Diapers', 'Bread'}
{}
this is F
[[]]


Now Functions to generate Rules

In [43]:
def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.01):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.
    """
    m = len(H[0])
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.01, VERBOSE=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A frequent itemset.

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:

            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if VERBOSE:
                print( "" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence, VERBOSE=True):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([item]) for item in freq_set]
            print(H1)
            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, VERBOSE=VERBOSE)

    return rules

Now test the Genration of Rules

In [44]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.001, VERBOSE=True)

In [45]:
print(H)

[]
